# Réseaux à partir des thèses soutenues en France

Un jeu de données public et disponible : https://www.data.gouv.fr/datasets/theses-soutenues-en-france-depuis-1985/

Trois étapes :

- Découvrir le jeu de données et filtrer un sous-ensemble
- Construire un réseau de co-direction
- Analyser le réseau de co-direction

## Explorer le jeu de données

Astuce : ne charger qu'une partie si besoin au moins au début pour faire le traitement

In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("theses-soutenues.csv", low_memory=False)
df.shape

In [5]:
df.head()

,accessible,auteur.idref,auteur.nom,auteur.prenom,cas,code_etab,date_soutenance,directeurs_these.0.idref,directeurs_these.0.nom,directeurs_these.0.prenom,...,sujets_rameau.7,sujets_rameau.8,sujets_rameau.9,these_sur_travaux,titres.autre.0,titres.autre.1,titres.autre.2,titres.autre.3,titres.en,titres.fr
0,non,035294140,Quartararo,Juliette,NaN,PA06,1996-01-01,130245933,Kasztelan,Slavik,...,Sulfures métalliques,Catalyseurs -- Analyse,Hydrotraitement,non,NaN,NaN,NaN,NaN,NaN,Reactivite de sulfures mixtes supportes non co...
1,non,132450372,Kyselková,Martina,NaN,LYO1,2008-01-01,124874657,Moënne-Loccoz,Yvan,...,NaN,NaN,NaN,non,NaN,NaN,NaN,NaN,Caractérisation par puce à ADN taxonomique de ...,NaN
2,non,122277732,Skrzypczak,Alexandre,NaN,REN1,2007-01-01,068762852,Palicot,Jacques,...,NaN,NaN,NaN,non,NaN,NaN,NaN,NaN,Contribution to the study of the OFDM / OQAM a...,Contribution à l'étude des modulations multipo...
3,non,NaN,AMARGER,VERONIQUE,NaN,PA07,1993-01-01,089062388,L'Hoir,Alain,...,NaN,NaN,NaN,non,NaN,NaN,NaN,NaN,Optimization and comparison of self-aligned te...,Optimisation et comparaison de technologies au...
4,non,197128149,Catanzariti,Jean François,NaN,LIL2,1994-01-01,059910968,Delcambre,Bernard,...,NaN,NaN,NaN,non,NaN,NaN,NaN,NaN,NaN,Les troubles oculomoteurs dans la scoliose idi...


In [1]:
#list(df.columns)

Réduire le jeu de données 

- Après les années 2000
- uniquement sur certaines thématiques (sociologie)

Ajouter une feature année et réduire

In [6]:
df["annee"] = df["date_soutenance"].apply(lambda x: None if x is None else str(x).split("-")[0])
df["annee"] = pd.to_numeric(df["annee"], errors='coerce')
df_ss = df[df["annee"] > 2000]

Construire la colonne des co-directions

In [7]:
liste_dir = [f"directeurs_these.{i}.idref" for i in range(0, 7)]
df_ss["id_dir"] = df_ss[liste_dir].apply(lambda x: x.dropna().tolist(), axis=1)

/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_51617/617968191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ss["id_dir"] = df_ss[liste_dir].apply(lambda x: x.dropna().tolist(), axis=1)


Construire un dictionnaire id > nom

In [16]:
dic_dir = df.drop_duplicates(subset=["directeurs_these.0.idref"])[["directeurs_these.0.idref","directeurs_these.0.nom","directeurs_these.0.prenom"]]
dic_dir = dic_dir.set_index("directeurs_these.0.idref")
dic_dir["directeur"] = dic_dir["directeurs_these.0.nom"] + "_" + dic_dir["directeurs_these.0.prenom"]
dic_dir = dic_dir["directeur"].to_dict()

Filtrer uniquement sur une discipline

In [8]:
f = df["discipline"].str.lower().str.contains("sociol")
df_ss2 = df_ss[f]

/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_51617/2292188866.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ss2 = df_ss[f]


In [12]:
df_ss2.head(
)

,accessible,auteur.idref,auteur.nom,auteur.prenom,cas,code_etab,date_soutenance,directeurs_these.0.idref,directeurs_these.0.nom,directeurs_these.0.prenom,...,sujets_rameau.9,these_sur_travaux,titres.autre.0,titres.autre.1,titres.autre.2,titres.autre.3,titres.en,titres.fr,annee,id_dir
36,non,138977119,Aires Filho,Benaias,NaN,EHES,2003-01-01,026960877,Lagrave,Rose-Marie,...,NaN,non,NaN,NaN,NaN,NaN,One example of conservative modernization : th...,Un exemple de modernisation conservatrice : l'...,2003.0,[026960877]
347,non,057215979,Courtebras,Bernard,NaN,LYO2,2005-01-01,027187152,Vincent,Guy,...,NaN,non,NaN,NaN,NaN,NaN,NaN,Socialisation et performances mathématiques : ...,2005.0,[027187152]
399,non,158358953,Bob,Ibrahima,NaN,AMIE,2009-01-01,027011119,Marouf,Nadir,...,NaN,non,NaN,NaN,NaN,NaN,Urban women's participative strategies in the ...,Les stratégies participatives des femmes urbai...,2009.0,[027011119]
494,non,05986320X,Doumenc Sakir,Colette,NaN,STR2,2001-01-01,028296591,Guth,Suzie,...,NaN,non,NaN,NaN,NaN,NaN,Education and social stratification : the Hind...,Education et stratification sociale : les comm...,2001.0,[028296591]
737,non,130180726,Diop,Moussa,NaN,PA09,2008-01-01,029291232,Trinh,Sylvaine,...,NaN,non,NaN,NaN,NaN,NaN,"Water and development : scales, temporalities,...","Eau et Développement : Échelles, temporalités,...",2008.0,[029291232]


Sauvegarder le dataframe

In [11]:
df_ss2.to_csv("theses_soutenues_sup2000_socio.csv", index=False)

## Construire le réseau

In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import itertools

In [13]:
G = nx.Graph()
for _, row in df_ss2.iterrows():
    for dir in row["id_dir"]:
        dir = str(dir)
        if not G.has_node(dir):
            G.add_node(dir, label=dir, weight=1)
        else:
            G.nodes[dir]["weight"] += 1
    for i,j in itertools.combinations(row["id_dir"], 2):
        i = str(i)
        j = str(j)
        if G.has_edge(i, j):
            G[i][j]["weight"] += 1
        else:
            G.add_edge(i, j, weight=1)

In [14]:
print(G)

Graph with 1899 nodes and 1092 edges


Ajouter des attributs

In [17]:
nx.set_node_attributes(G, dic_dir, "directeur")

Représenter une partie du réseau

In [19]:
from ipysigma import Sigma
Sigma(G, node_size=G.degree, edge_size="weight", node_label="directeur")

Sigma(nx.Graph with 1,899 nodes and 1,092 edges)

## Analyser le réseau

- Avoir des statistiques générales du réseaux ?
- Qui sont les directeurs de thèse les plus centraux ?
- Quelle est la plus grande composante du réseau ?